In [2]:
!pip install numpy 
!pip install pandas 
!pip install tensorflow
!pip install matplotlib
!pip install keras
!pip install keras

In [3]:
!pip install scikit-learn

In [5]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt


In [6]:
#
data=pd.read_csv(r"C:\Users\Aayushi\Downloads\stock.csv")

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7160 entries, 0 to 7159
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       7160 non-null   object 
 1   Open       7160 non-null   float64
 2   High       7160 non-null   float64
 3   Low        7160 non-null   float64
 4   Close      7160 non-null   float64
 5   Adj Close  7160 non-null   float64
 6   Volume     7160 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 391.7+ KB


In [8]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,01-11-1994,8.3125,8.3125,8.1875,8.2500,6.565471,2176400
1,02-11-1994,8.2500,8.2500,8.1875,8.1875,6.515731,2348200
2,03-11-1994,8.1875,8.2500,8.1250,8.1875,6.515731,2060800
3,04-11-1994,8.1875,8.2500,8.1250,8.1250,6.465993,2663800
4,07-11-1994,8.0625,8.0625,8.0000,8.0625,6.416255,1049400


In [9]:
data["Close"]=pd.to_numeric(data.Close,errors='coerce')
data=data.dropna()

In [10]:
trainData=data.iloc[:,4:5].values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7160 entries, 0 to 7159
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       7160 non-null   object 
 1   Open       7160 non-null   float64
 2   High       7160 non-null   float64
 3   Low        7160 non-null   float64
 4   Close      7160 non-null   float64
 5   Adj Close  7160 non-null   float64
 6   Volume     7160 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 391.7+ KB


In [11]:
sc=MinMaxScaler(feature_range=(0,1))
trainData=sc.fit_transform(trainData)
trainData.shape

(7160, 1)

In [12]:
X_train=[]
Y_train=[]

for i in range (60,1149):
    X_train.append(trainData[i-60:i,0])
    Y_train.append(trainData[i,0])

X_train,Y_train=np.array(X_train),np.array(Y_train)

In [13]:
X_train=np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
X_train.shape

(1089, 60, 1)

In [14]:
model=Sequential()
model.add(LSTM(units=100,return_sequences=True, input_shape=(X_train.shape[1],1)))
model.add(Dropout(0.2))

model.add(LSTM(units=100,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=100,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=100,return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(units=1))
model.compile(optimizer='adam', loss="mean_squared_error")

In [ ]:
hist=model.fit(X_train,Y_train,epochs=20,batch_size=32,verbose=2)

Epoch 1/20
35/35 - 24s - loss: 5.9675e-05 - 24s/epoch - 675ms/step
Epoch 2/20
35/35 - 8s - loss: 2.1741e-05 - 8s/epoch - 227ms/step
Epoch 3/20
35/35 - 8s - loss: 1.3021e-05 - 8s/epoch - 215ms/step
Epoch 4/20
35/35 - 7s - loss: 6.9659e-06 - 7s/epoch - 201ms/step
Epoch 5/20
35/35 - 7s - loss: 3.5715e-06 - 7s/epoch - 194ms/step
Epoch 6/20
35/35 - 7s - loss: 5.5222e-06 - 7s/epoch - 188ms/step
Epoch 7/20
35/35 - 7s - loss: 3.8996e-06 - 7s/epoch - 194ms/step
Epoch 8/20
35/35 - 6s - loss: 3.0202e-06 - 6s/epoch - 175ms/step
Epoch 9/20
35/35 - 6s - loss: 1.8460e-06 - 6s/epoch - 182ms/step
Epoch 10/20
35/35 - 7s - loss: 2.8164e-06 - 7s/epoch - 199ms/step
Epoch 11/20
35/35 - 7s - loss: 3.0257e-06 - 7s/epoch - 195ms/step
Epoch 12/20


In [ ]:
plt.plot(hist.history['loss'])
plt.title('Training model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'],loc='upper left')
plt.show()

In [ ]:
#
testData=pd.read_csv(r"C:\Users\Aayushi\Downloads\stock.csv")
testData["Close"]=pd.to_numeric(testData.Close,errors='coerce')
testData=testData.dropna()
testData=testData.iloc[:,4:5]
Y_test=testData.iloc[60:,0:].values
#VALUES 
inputClosing=testData.iloc[:,0:].values
inputClosing_scaled=sc.transform(inputClosing)
inputClosing_scaled.shape
X_test=[]
length = len(testData)
time=60
for i in range(time,length):
    X_test.append(inputClosing_scaled[i-time:i,0])
X_test=np.array(X_test)
X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
X_test.shape

In [ ]:
y_pred=model.predict(X_test)
y_pred

In [ ]:
predicted_price=sc.inverse_transform(y_pred)

In [ ]:
plt.plot(Y_test,color='red',label='Actual Stock  Price')
plt.plot(predicted_price,color='yellow',label='Predicetd Stock Price')
plt.title('Stock Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()